In [4]:
import pickle
import heapq
import math
import folium

# Load the updated graph data
with open("graph_simplified_id.pickle", "rb") as f:
    G = pickle.load(f)

# Haversine formula to calculate Euclidean (great-circle) distance between two latitude/longitude points
def haversine_distance(coord1, coord2):
    R = 6371000  # Earth radius in meters
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c  # Distance in meters

# A* search algorithm using travel time as the cost and Euclidean distance as the heuristic
def a_star_search(graph, start, goal):
    # Priority queue (min-heap), stores tuples of (estimated_total_cost, current_node, cost_from_start, path)
    open_list = []
    heapq.heappush(open_list, (0, start, 0, [start]))
    
    # A dictionary to store the shortest known cost from start to every node
    g_costs = {start: 0}
    
    # A set to track visited nodes
    visited = set()
    
    # Get the coordinates of the goal node
    goal_coordinates = (graph[goal]['stop_latitude'], graph[goal]['stop_longitude'])

    while open_list:
        # Get the node with the lowest estimated total cost (f(n) = g(n) + h(n))
        _, current_node, current_cost, path = heapq.heappop(open_list)
        
        # If we reached the goal, return the path and the cost
        if current_node == goal:
            return path, current_cost
        
        # Skip if we've already visited this node
        if current_node in visited:
            continue
        visited.add(current_node)
        
        # Get the coordinates of the current node
        current_node_coordinates = (graph[current_node]['stop_latitude'], graph[current_node]['stop_longitude'])
        
        # Explore neighbors
        for neighbor, neighbor_info in graph[current_node]['neighbors'].items():
            neighbor_coordinates = (graph[neighbor]['stop_latitude'], graph[neighbor]['stop_longitude'])
            
            # Calculate the cost to move from the current node to the neighbor (travel time)
            travel_time = neighbor_info['travel_time']
            
            # New g(n): cost from the start to this neighbor (travel time)
            new_g_cost = current_cost + travel_time
            
            # If this path is better, or the neighbor hasn't been visited yet
            if neighbor not in g_costs or new_g_cost < g_costs[neighbor]:
                g_costs[neighbor] = new_g_cost
                
                # Heuristic: straight-line distance to goal (h(n))
                h_cost = haversine_distance(neighbor_coordinates, goal_coordinates)
                
                # Total estimated cost (f(n) = g(n) + h(n))
                f_cost = new_g_cost + h_cost
                
                # Add the neighbor to the priority queue
                heapq.heappush(open_list, (f_cost, neighbor, new_g_cost, path + [neighbor]))
    
    return None, float('inf')  # Return None if no path exists

# Function to extract coordinates of the stops in the path
def get_coordinates(graph, path):
    coordinates = []
    for stop in path:
        # Extract coordinates from the graph
        coordinates.append((graph[stop]['stop_latitude'], graph[stop]['stop_longitude']))
    return coordinates

# Function to create a Folium map and visualize the path
def visualize_path(graph, path):
    # Get the coordinates of the path
    path_coords = get_coordinates(graph, path)
    
    # If no coordinates were found, return an error
    if not path_coords:
        print("Could not find coordinates for the path")
        return
    
    # Explicitly center the map on the first stop (Ropsten) and specify the zoom level
    start_coords = path_coords[0]
    map_center = [start_coords[0], start_coords[1]]
    map_obj = folium.Map(location=map_center, zoom_start=14)  # Increased zoom level for better visibility
    
    # Highlight the starting point (Ropsten)
    folium.Marker(
        location=[path_coords[0][0], path_coords[0][1]],
        popup=f"Start: {path[0]} (Ropsten)",
        icon=folium.Icon(color='green', icon='play', prefix='fa')  # Green marker for start
    ).add_to(map_obj)
    
    # Add markers for intermediate stops (if any)
    for idx, stop_coords in enumerate(path_coords[1:-1], start=1):
        folium.Marker(
            location=[stop_coords[0], stop_coords[1]],
            popup=f"Stop {idx + 1}: {path[idx]}",
            icon=folium.Icon(color='blue')
        ).add_to(map_obj)

    # Highlight the ending point (Nybroplan)
    folium.Marker(
        location=[path_coords[-1][0], path_coords[-1][1]],
        popup=f"End: {path[-1]} (Nybroplan)",
        icon=folium.Icon(color='red', icon='flag', prefix='fa')  # Red marker for end
    ).add_to(map_obj)
    
    # Extract latitude and longitude for the PolyLine
    polyline_coords = [(coord[0], coord[1]) for coord in path_coords]
    
    # Add the PolyLine to visualize the route on the map
    folium.PolyLine(polyline_coords, color="red", weight=5, opacity=0.7).add_to(map_obj)
    
    # Display the map
    return map_obj

# Example usage with the path from A* search
path, total_travel_cost = a_star_search(G, "Ropsten", "Nybroplan")

print(f"Shortest path: {path}")
print(f"Total travel time: {total_travel_cost:.2f} seconds")

# Visualize the path
map_obj = visualize_path(G, path)

# Save map to HTML and display
map_obj.save('shortest_path_map.html')
map_obj

Shortest path: ['Ropsten', 'Frihamnen', 'Lidingö/Dalénum', 'Nacka strand', 'Nybroplan']
Total travel time: 44460.00 seconds
